## Demo Webcam acquisition

In [0]:
# Imports
from cv2 import *    ; print(f'  OpenCV          ver: {cv2.__version__}')
# import cv2


  OpenCV          ver: 4.1.2


In [0]:
# Settings


## Tools - Images

In [0]:
# Tools

# Plaguarised from https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=buJCl90WhNfq
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def acquire_webcam_image_to_file(filename='photo.jpg', quality=0.8):

  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Image';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);
      //await new Promise((resolve) => document.onload = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data_jpeg = eval_js('takePhoto({})'.format(quality))
  bytes_jpeg = b64decode(data_jpeg.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(bytes_jpeg)
  return filename

def save_image_to_file():
    pass

def get_image_from_file():
    image = Image(filename)
    return image

def display_image():
    pass

# Test
if False:
    from IPython.display import Image
    try:
        # Acquire from webcam
        filename = acquire_webcam_image_to_file()
        print(f'Saved to {filename}')
        
        # Show the image which was just taken.
        #display(Image(filename))

    except Exception as err:
        # Errors will be thrown if the user does not have a webcam or if they do not
        # grant the page permission to access it.
        print(str(err))


## Tools - CNN

In [0]:

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

# Load the trained VGG16 model
model = VGG16()

def get_class_from_image(filename):

    # Load image from file
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array, reshape, and normalise
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #image = preprocess_input(image)

    # Predict probabilities across all output classes
    yhat = model.predict(image)
    labels = decode_predictions(yhat)  #; print(labels)

    # print the classification
    most_likely_class_label = labels[0][0]
    #print(f'{most_likely_class_label[1]} ({most_likely_class_label[2]*100:.3f}%)' )

    return most_likely_class_label[1], most_likely_class_label[2]
                                        
# Test
if False:
    class_name, probability = get_class_from_image('photo.jpg')
    print(f'{class_name} ({probability*100:.2f}%)' )

## Main

In [0]:
# Main
from IPython.display import Image, clear_output
import time

# Load CNN

# Loop displaying image
while True:

    try:
        # Acquire image
        filename = acquire_webcam_image_to_file()  #; print(f'Saved to {filename}')
        display(Image(filename))

        # Identify
        class_name, probability = get_class_from_image(filename)
        # print(f'{class_name} ({probability*100:.2f}%)' )
        print(f'Most likely:\n\n    {class_name}     ({probability*100:.2f}%)\n' )

    except Exception as err:
        # Errors will be thrown if the user does not have a webcam or if they do not
        # grant the page permission to access it.
        print(str(err))

    # WWait and clear output cell
    time.sleep(3)
    clear_output()





<IPython.core.display.Javascript object>

## Scratch